In [1]:
from skr_web_api import Submission, METAMAP_INTERACTIVE_URL
import pandas as pd
import numpy as np
import regex as re
import argparse
import glob
from tqdm import tqdm

In [9]:
METAMAP_INTERACTIVE_URL


'https://ii.nlm.nih.gov/cgi-bin/II/UTS_Required/API_MM_interactive.pl'

In [2]:
email = 'dmd2225@cumc.columbia.edu'
apikey = '187a5701-83da-4679-9975-c46c1398d525'
inst = Submission(email, apikey=apikey)
inst.set_serviceurl(METAMAP_INTERACTIVE_URL)
#inst.set_serviceurl('https://ii.nlm.nih.gov/Interactive/UTS_Required/MetaMap.html')
inst.set_mm_ksource('2022AA')

In [4]:
df = pd.read_csv('discharge_cleaner.csv', parse_dates=['charttime', 'storetime'])
df = df.dropna()
df
# sort the 

,subject_id,note_id,hadm_id,charttime,storetime,text,corpus
0,17144872,17144872-DS-3,21926905,2211-07-21,2211-07-21 21:45:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\nA ___ gentleman...
1,13184296,13184296-DS-3,28785633,2211-05-07,2211-05-09 09:40:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\n___ yo ___ spea...
2,15566010,15566010-DS-10,29192759,2210-09-11,2210-09-12 15:03:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\n___ with suspec...
3,12953653,12953653-DS-6,26851914,2210-09-06,2210-09-11 20:46:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\nTime (and date)...
4,13774741,13774741-DS-10,27575074,2210-07-23,2210-07-24 23:12:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\n___ y/o F with ...
...,...,...,...,...,...,...,...
145901,18106347,18106347-DS-15,24305596,2110-01-15,2110-01-15 11:25:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\nMs. ___ is a __...
145902,11274131,11274131-DS-7,28390257,2110-01-14,2110-01-14 18:43:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\n___ year-old le...
145903,15918556,15918556-DS-20,27863083,2110-01-14,2110-01-14 13:31:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\nHPI: The patien...
145904,15350437,15350437-DS-11,20383396,2110-01-12,2110-01-13 07:13:00,Name Unit Admission Date Discharge Date Da...,History of Present Illness:\n\nThe patient is ...


In [8]:
texts = df['text'].values
pids = df['subject_id'].values
with open('metamap1.txt', 'w', encoding='utf-8', errors='replace') as f:
    for pid, text in zip(pids, texts):
        #text = text.decode('utf-8')
        f.write(str(pid)+ '|'+text + '\n')
        break
    

In [10]:
inst.init_generic_batch('metamap', "--negex -y -s -I -J dsyn,sosy")
inst.set_batch_file('metamap', text)
result = inst.submit()
result.content

In [4]:
corpus = df.text[0]
print(corpus)

Name  Unit  Admission Date  Discharge Date  Date Birth  Sex  Service  UROLOGY Allergies  Known Allergies / Adverse Drug Reactions Attending  Chief Complaint  erectile dysfunction Major Surgical Invasive Procedure  Insertion inflatable penile prosthesis . History Present Illness  gentleman known moderate COPD , history DVT PE , anticoagulation . symptoms signs infection , anemia , recurrent PE angina/CHF . increased risk given long tobacco history known COPD , well history DVT PE medically cleared surgery . EKG obtained notable sinus bradycardia  axis + , PR interval . , QRS . , QT interval . . acute ischemic ST T-wave changes appreciated . medically cleared surgery . Risk slightly increased due COPD , manageable Anesthesia . Past Medical History  PAST MEDICAL HISTORY  notable history DVT PE long tobacco history , quit  weeks COPD , history colorectal cancer , prothrombin mutation , varicocele , dilatation distal aorta degenerative joint disease . also basal squamous cell carcinomas . S

We are extracting present illness and hospital course:<br>
“History of Present Illness:” to “Past Medical History:”<br>
“Brief Hospital Course:” to “ Medications on Admission:”<br>
So maybe change the codes to<br>
match = re.search("History of Present Illness:(.*?)Past Medical History:", df['text'][0], re.DOTALL)<br>
match = re.search("Brief Hospital Course:(.*?)Medications on Admission:", df['text'][0], re.DOTALL)

In [4]:
import re

def regex_match(text):
    extract = ''
    match1 = re.search("History of Present Illness:(.*?)Past Medical History:", text, re.DOTALL)
    match2 = re.search("Brief Hospital Course:(.*?)Medications on Admission:", text, re.DOTALL)
    if match1:
        extract = 'History of Present Illness:\n\n'+ match1.group(1).strip()
    if match2:
        if extract:
            extract += '\n\n'
        extract += 'Brief Hospital Course:\n\n' + match2.group(1).strip()
    if not extract:
        extract = None
    return extract


In [40]:
df['corpus'] = df.text.apply(regex_match)

In [42]:
df['corpus'].value_counts(dropna=False)

corpus
None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [57]:
print(df[df['corpus'].isna()].text.values[1])

 
Name:  ___                ___ No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: ORTHOPAEDICS
 
Allergies: 
Flagyl
 
Attending: ___.
 
Chief Complaint:
Needs rehab placement
 
Major Surgical or Invasive Procedure:
R DHS for an intertrochanteric hip fracture, performed in ___ 
on ___

 
History of Present Illness:
___ F presents as walk in to clinic direct from airport after 
undergoing R DHS in a ___ on ___ for a R hip 
IT fracture sustained during a fall.  Pt reports se is unable to 
ambulate or perform self care. She is wheelchair bound. Taking 
Lovenox since surgery.  She had LENIs per report, which were 
negative.

PMH
Osteoporosis
Arthritis
L femur fx / ___, ___
s/p hysterectomy

 
Brief Hospital Course:
The patient presented as a same day admission for rehab 
placement. The patient was given anticoagulation per routine. 
The patient's home medications were continued throughout this 
hospitalization. The pati

In [44]:
df = df.sort_values(by='storetime', ascending=False)
df = df.reset_index(drop=True)
df.to_csv('discharge_clean.csv', index=False)

In [62]:
df_train = df.loc[0:int(len(df)*0.8)]
df_train

,subject_id,note_id,hadm_id,charttime,storetime,text,corpus
0,17144872,17144872-DS-3,21926905,2211-07-21,2211-07-21 21:45:00,\nName: ___ Unit No: __...,History of Present Illness:\n\nA ___ gentleman...
1,13184296,13184296-DS-3,28785633,2211-05-07,2211-05-09 09:40:00,\nName: ___ Unit No: __...,History of Present Illness:\n\n___ yo ___ spea...
2,15566010,15566010-DS-10,29192759,2210-09-11,2210-09-12 15:03:00,\nName: ___ Unit No: ___\n ...,History of Present Illness:\n\n___ with suspec...
3,12953653,12953653-DS-6,26851914,2210-09-06,2210-09-11 20:46:00,\nName: ___ Unit No: ___...,History of Present Illness:\n\nTime (and date)...
4,13774741,13774741-DS-10,27575074,2210-07-23,2210-07-24 23:12:00,\nName: ___ Unit No: ___\n \...,History of Present Illness:\n\n___ y/o F with ...
...,...,...,...,...,...,...,...
116720,10237315,10237315-DS-5,22558185,2128-10-07,2128-10-09 16:49:00,\nName: ___ Unit No: ___...,History of Present Illness:\n\n___ y.o. M with...
116721,10049095,10049095-DS-15,22362949,2128-10-09,2128-10-09 15:56:00,\nName: ___ Unit No: ___\n...,History of Present Illness:\n\nMr. ___ is a __...
116722,10494276,10494276-DS-10,24085567,2128-10-08,2128-10-09 14:04:00,\nName: ___ Unit No: __...,History of Present Illness:\n\n___ w/ bladder ...
116723,16426261,16426261-DS-12,21108853,2128-10-09,2128-10-09 13:42:00,\nName: ___ Unit No: ___\n \...,History of Present Illness:\n\nPatient is a __...


In [28]:
def to_metamap(text, inst):
    inst.init_mm_interactive(text, ksource='2022AA', args='-I -J dsyn,sosy')
    return inst.submit()
    

## Randomly Extracting 100 rows

```df_100 = df.sample(n=100)
for i, row in df_100.iterrows():
    patient_id = row['subject_id']
    corpus = row['corpus']
    #corpus = corpus.encode('utf-8').decode('utf-8', 'replace')
    with open(f'./data/{patient_id}.txt', 'w', encoding='utf-8') as f:
        f.write(corpus)```

In [29]:
ann_100 = glob.glob('data/*.txt')
for ann in tqdm(ann_100):
    patient_id = ann.split('\\')[-1].split('.')[0]
    with open(ann, 'r') as f:
        text = f.read()
        output = to_metamap(text, inst)
        with open(f'./data/{patient_id}.json', 'w') as f:
            f.write(output.content.decode())

100%|██████████| 100/100 [12:27<00:00,  7.48s/it]


In [60]:
df_100

,subject_id,note_id,hadm_id,charttime,storetime,text,corpus
133452,13174125,13174125-DS-6,24243315,2118-11-18,2118-11-21 20:37:00,\nName: ___ Unit No: __...,History of Present Illness:\n\nMs. ___ is a lo...
5037,17417527,17417527-DS-10,29309659,2190-02-13,2190-02-13 14:20:00,\nName: ___ Unit No: ___...,History of Present Illness:\n\n___ yo RHM on C...
113640,15801498,15801498-DS-14,23847863,2130-06-19,2130-06-19 12:40:00,\nName: ___ Unit No: ___\n...,History of Present Illness:\n\nMr. ___ is an _...
9766,10306798,10306798-DS-14,25349666,2187-05-31,2187-05-31 15:06:00,\nName: ___ Unit No: ___\...,History of Present Illness:\n\nThe patient is ...
48620,17788710,17788710-DS-12,25689558,2165-12-27,2165-12-27 12:59:00,\nName: ___ Unit No: ___\n ...,History of Present Illness:\n\nMr. ___ is a __...
...,...,...,...,...,...,...,...
133645,12359924,12359924-DS-4,29621134,2118-10-07,2118-10-07 15:05:00,\nName: ___ Unit No: ___\n...,History of Present Illness:\n\nThis is a ___ y...
112540,15866824,15866824-DS-10,28493213,2131-01-24,2131-01-24 10:37:00,\nName: ___ Unit No: __...,History of Present Illness:\n\nMr. ___ is a __...
27710,10177158,10177158-DS-7,23456006,2177-07-23,2177-07-23 21:11:00,\nName: ___ Unit No: ___\...,History of Present Illness:\n\n___ year old __...
42505,17489409,17489409-DS-14,21493203,2169-05-27,2169-05-27 12:19:00,\nName: ___. Unit No: ___\n ...,History of Present Illness:\n\nseen as consult...


In [67]:
email = 'dmd2225@cumc.columbia.edu'
apikey = '19895da9-5d2d-41da-80a3-26a6e3b55fdf'
inst = Submission(email, apikey=apikey)
inst.set_mm_ksource('2022AA')
for i, row in df_100.iterrows():
    patient_id = row['subject_id']
    corpus = row['corpus']

    inst.init_mm_interactive(row['corpus'], ksource='2022AA', args='-I --negex -J dsyn sosy')
    response = inst.submit()
    print(f'response status: {response.status_code}')
    with open(f'./data/{patient_id}.json', 'w') as f:
        f.write(response.content.decode())
    #(f'content:\n {response.content.decode()}')

response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response status: 200
response stat

In [10]:
email = 'dmd2225@cumc.columbia.edu'
apikey = '19895da9-5d2d-41da-80a3-26a6e3b55fdf'
input_file = 'extract.txt'
inst = Submission(email, apikey=apikey)
inst.set_mm_ksource('2022AA')
inst.init_mm_interactive(extract)
response = inst.submit()
print(f'response status: {response.status_code}')
print(f'content:\n {response.content.decode()}')

response status: 200
content:
 /dmzfiler/II_Group/MetaMap2020/public_mm/bin/SKRrun.20 /dmzfiler/II_Group/MetaMap2020/public_mm/bin/metamap20.BINARY.Linux --lexicon db -Z 2020AB --silent -N
USER|MMI|33.49|Deny (action)|C2700401|[acty]|["Deny"-tx-13-"denies"-verb-0,"Deny"-tx-12-"denies"-verb-0,"Deny"-tx-9-"denies"-verb-0,"Deny"-tx-8-"denies"-verb-0,"Deny"-tx-6-"denies"-verb-0,"Deny"-tx-5-"denies"-verb-0]|TX|1132/6;1089/6;811/6;667/6;509/6;465/6|
USER|MMI|14.88|Endocrine System Findings Domain|C3538926|[inpr]|["ED"-tx-14-"ED"-noun-0,"ED"-tx-11-"ED"-noun-0,"ED"-tx-10-"ED"-noun-0,"ED"-tx-1-"ED"-noun-0]|TX|1211/2;982/2;929/2;94/2|
USER|MMI|11.92|Regulatory Report|C3273238|[inpr]|["Report"-tx-13-"reports"-verb-0,"Report"-tx-11-"report"-noun-0,"Report"-tx-2-"reports"-verb-0]|TX|1162/7;985/6;149/7|
USER|MMI|11.92|Report (document)|C0684224|[inpr]|["Reports"-tx-13-"reports"-verb-0,"Report"-tx-11-"report"-noun-0,"Reports"-tx-2-"reports"-verb-0]|TX|1162/7;985/6;149/7|
USER|MMI|11.92|Report:Finding